In [1]:
# No leak data
import pandas as pd
import numpy as np
import gc 
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import datetime


In [2]:
# Bayesian Optimization Functions

# https://www.kaggle.com/aitude/ashrae-hyperparameter-tuning

In [3]:
train_df = pd.read_pickle('train_df.pkl')
test_df = pd.read_pickle('test_df.pkl')

In [4]:
# some feature enginnering

train_df['age'] = train_df['year_built'].max()-train_df['year_built']+1
test_df['age'] = test_df['year_built'].max() - test_df['year_built'] + 1

#Handling missing values
# To streamline this though process it is useful to know the 3 categories in which missing data can be classified into:

# Missing Completely at Random (MCAR)
# Missing at Random (MAR)
# Missing Not at Random (MNAR)

train_df['floor_count'] = train_df['floor_count'].fillna(-999).astype(np.int16)
test_df['floor_count'] = test_df['floor_count'].fillna(-999).astype(np.int16)

train_df['year_built'] = train_df['year_built'].fillna(-999).astype(np.int16)
test_df['year_built'] = test_df['year_built'].fillna(-999).astype(np.int16)

train_df['age'] = train_df['age'].fillna(-999).astype(np.int16)
test_df['age'] = test_df['age'].fillna(-999).astype(np.int16)

train_df['cloud_coverage'] = train_df['cloud_coverage'].fillna(-999).astype(np.int16)
test_df['cloud_coverage'] = test_df['cloud_coverage'].fillna(-999).astype(np.int16) 

train_df['square_feet'] = np.log1p(train_df['square_feet'])
test_df['square_feet'] = np.log1p(test_df['square_feet'])

In [5]:
def make_is_bad_zero(Xy_subset, min_interval=48, summer_start=3000, summer_end=7500):
    """Helper routine for 'find_bad_zeros'.
    
    This operates upon a single dataframe produced by 'groupby'. We expect an 
    additional column 'meter_id' which is a duplicate of 'meter' because groupby 
    eliminates the original one."""
    meter = Xy_subset.meter_id.iloc[0]
    is_zero = Xy_subset.meter_reading == 0
    if meter == 0:
        # Electrical meters should never be zero. Keep all zero-readings in this table so that
        # they will all be dropped in the train set.
        return is_zero

    transitions = (is_zero != is_zero.shift(1))
    all_sequence_ids = transitions.cumsum()
    ids = all_sequence_ids[is_zero].rename("ids")
    if meter in [2, 3]:
        # It's normal for steam and hotwater to be turned off during the summer
        keep = set(ids[(Xy_subset.timestamp < summer_start) |
                       (Xy_subset.timestamp > summer_end)].unique())
        is_bad = ids.isin(keep) & (ids.map(ids.value_counts()) >= min_interval)
    elif meter == 1:
        time_ids = ids.to_frame().join(Xy_subset.timestamp).set_index("timestamp").ids
        is_bad = ids.map(ids.value_counts()) >= min_interval

        # Cold water may be turned off during the winter
        jan_id = time_ids.get(0, False)
        dec_id = time_ids.get(8283, False)
        if (jan_id and dec_id and jan_id == time_ids.get(500, False) and
                dec_id == time_ids.get(8783, False)):
            is_bad = is_bad & (~(ids.isin(set([jan_id, dec_id]))))
    else:
        raise Exception(f"Unexpected meter type: {meter}")

    result = is_zero.copy()
    result.update(is_bad)
    return result

def find_bad_zeros(X, y):
    """Returns an Index object containing only the rows which should be deleted."""
    Xy = X.assign(meter_reading=y, meter_id=X.meter)
    is_bad_zero = Xy.groupby(["building_id", "meter"]).apply(make_is_bad_zero)
    return is_bad_zero[is_bad_zero].index.droplevel([0, 1])

def find_bad_sitezero(X):
    """Returns indices of bad rows from the early days of Site 0 (UCF)."""
    return X[(X.timestamp < 3378) & (X.site_id == 0) & (X.meter == 0)].index

def find_bad_building1099(X, y):
    """Returns indices of bad rows (with absurdly high readings) from building 1099."""
    return X[(X.building_id == 1099) & (X.meter == 2) & (y > 3e4)].index

def find_bad_rows(X, y):
    return find_bad_zeros(X, y).union(find_bad_sitezero(X)).union(find_bad_building1099(X, y))


In [6]:
# bad rows 삭제 3% 
train_df['timestamp'] = (train_df.timestamp - pd.to_datetime("2016-01-01")).dt.total_seconds() // 3600
rows = find_bad_rows(train_df.drop(['meter_reading'],axis =1 ),train_df['meter_reading'])
rows = list(rows)
train_df= train_df.drop(rows)

In [7]:
# drop_cols = ['date',"precip_depth_1_hr", "sea_level_pressure", "wind_direction", "wind_speed","timestamp"]
drop_cols = ['date',"timestamp"]
target = train_df["meter_reading_log1p"]
del train_df["meter_reading"], train_df['meter_reading_log1p']
train_df = train_df.drop(drop_cols, axis=1)
drop_cols += ["row_id"]
# drop_cols.remove('date')
test_df = test_df.drop(drop_cols, axis=1)

In [8]:
train_df.head()
categorical_features = ["building_id", "site_id", "meter", "hour", "weekend",'month','is_holiday','primary_use']

In [9]:
leak_df = pd.read_pickle('leak_df.pkl')
leak_df = leak_df.dropna()


In [10]:
%%time
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error
from bayes_opt import BayesianOptimization
import pickle
folds = 3
seed = 99 #666

# Hyperparameter range
params_range = {
                'learning_rate' : (.01,.07),
                'max_depth' : (8,13),
                'subsample' : (0.7,0.9),
                'colsample_bytree' : (0.7,0.8),
                'reg_alpha' :( 0.1,0.4),
                'reg_lambda' : (0.5,0.85),
                
               }



Wall time: 796 ms


In [ ]:
%%time
paramlst = []
scorelst =[]

def running(learning_rate, max_depth, subsample, colsample_bytree, reg_alpha, reg_lambda):
    params = {
    'objective': 'reg:squarederror',
    'tree_method': 'gpu_hist',
    "learning_rate": learning_rate,
    "max_depth" : int(round(max_depth)),
    "subsample": max(min(subsample,1),0),    
    "colsample_bytree": max(min(colsample_bytree,1),0),
    "reg_alpha": max(min(reg_alpha,1),0),
    "reg_lambda": max(min(reg_lambda,1),0),
    "n_jobs" : 4,
    "missing" : -999
    }
    
    print(params)
    kf = KFold(n_splits=folds, shuffle=False, random_state=seed)
    scores = [] 
    models = []
    for i,(train_index, val_index) in enumerate(kf.split(train_df)):
        train_X = train_df.iloc[train_index]
    
        val_X = train_df.iloc[val_index]
        train_y = target.iloc[train_index]
        val_y = target.iloc[val_index]
        xgb_train = xgb.DMatrix(train_X, train_y)
        xgb_eval = xgb.DMatrix(val_X, val_y)
        gbm = xgb.train(params, xgb_train, num_boost_round = 3000,evals=[(xgb_train, 'train'), (xgb_eval, 'val')],
                    verbose_eval = 30 , early_stopping_rounds = 30)
        pickle.dump(gbm,open('xgb_opti_%s.pickle'%i, "wb") )
        gbm.__del__() 
        del xgb_train, xgb_eval
        gc.collect()
    for f in range(folds):
        models.append(pickle.load(open("xgb_opti_%s.pickle"%f, "rb")))
    
    ii = 0
    res3 = []
    step_size = 50000
        
    for j in tqdm(range(int(np.ceil(test_df.shape[0] / 50000)))):
        res3.append([np.expm1(model.predict(xgb.DMatrix(test_df.iloc[ii:ii + step_size]))) for model in models])
#         res3.append([np.expm1(model.predict(test_df.iloc[i:i + step_size])) for model in models])
        ii += step_size
        
    res4 = []
    print(len(res3))
    for k in range(len(res3)):
        res4.append((0.6*res3[k][0]+0.3*res3[k][1]+0.1*res3[k][2]))
#         if i==100 : break
    
    
    res4 = np.concatenate(res4)
    sample_submission = pd.read_csv('sample_submission.csv')
    sample_submission["meter_reading"] = res4
    sample_submission.loc[sample_submission['meter_reading'] < 0, 'meter_reading'] = 0

    sample_submission = pd.merge(sample_submission,leak_df.loc[:,['row_id','leak_meter_reading']],how ='inner')    
    score = np.sqrt(mean_squared_log_error( sample_submission['leak_meter_reading'], sample_submission['meter_reading'] ))
    print(score)
    scorelst.append(score)
    paramlst.append(params)
    
    df_para = pd.DataFrame({"parameter" : paramlst,"score": scorelst})
    df_para.to_csv('para_xgb.csv',index=False)
    
    return -np.sqrt(mean_squared_log_error( sample_submission['leak_meter_reading'], sample_submission['meter_reading'] ))        

    
Baysian = BayesianOptimization(running, params_range,random_state = seed)
Baysian.maximize(init_points = 3, n_iter = 20)

|   iter    |  target   | colsam... | learni... | max_depth | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------
{'objective': 'reg:squarederror', 'tree_method': 'gpu_hist', 'learning_rate': 0.03928470395443503, 'max_depth': 12, 'subsample': 0.8131234839221061, 'colsample_bytree': 0.7672278558630792, 'reg_alpha': 0.10943391628788945, 'reg_lambda': 0.7828174871776967, 'n_jobs': 4, 'missing': -999}


C:\Users\yseon\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	train-rmse:4.18461	val-rmse:4.14876
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 30 rounds.
[30]	train-rmse:1.63754	val-rmse:1.72808
[60]	train-rmse:0.977406	val-rmse:1.15648
[90]	train-rmse:0.801245	val-rmse:1.0255
[120]	train-rmse:0.727149	val-rmse:0.973967
[150]	train-rmse:0.688519	val-rmse:0.95315
[180]	train-rmse:0.658027	val-rmse:0.938277
[210]	train-rmse:0.63688	val-rmse:0.930689
[240]	train-rmse:0.621685	val-rmse:0.925372
[270]	train-rmse:0.607742	val-rmse:0.921826
[300]	train-rmse:0.59591	val-rmse:0.919714
[330]	train-rmse:0.587094	val-rmse:0.919248
[360]	train-rmse:0.577078	val-rmse:0.917668
[390]	train-rmse:0.568369	val-rmse:0.916735
[420]	train-rmse:0.561853	val-rmse:0.916068
[450]	train-rmse:0.55378	val-rmse:0.914992
[480]	train-rmse:0.54694	val-rmse:0.914141
[510]	train-rmse:0.542003	val-rmse:0.914152
[540]	train-rmse:0.535403	val-rmse:0.913339
[570]	train-rmse:0.529334	val-rmse:0.912

100%|████████████████████████████████████████████████████████████████████████████████| 834/834 [10:41<00:00,  1.31it/s]


834
0.9424573577507224
|  1        | -0.9425   |  0.7672   |  0.03928  |  12.13    |  0.1094   |  0.7828   |  0.8131   |
{'objective': 'reg:squarederror', 'tree_method': 'gpu_hist', 'learning_rate': 0.012801743230417197, 'max_depth': 13, 'subsample': 0.84935342019886, 'colsample_bytree': 0.7297622498703944, 'reg_alpha': 0.10204771991309657, 'reg_lambda': 0.7694275598664789, 'n_jobs': 4, 'missing': -999}


...

834
0.9392038184633338
|  21       | -0.9392   |  0.7138   |  0.01566  |  10.91    |  0.1315   |  0.8346   |  0.8995   |
{'objective': 'reg:squarederror', 'tree_method': 'gpu_hist', 'learning_rate': 0.010824085361533191, 'max_depth': 12, 'subsample': 0.8989735450992042, 'colsample_bytree': 0.7031129628046585, 'reg_alpha': 0.11553753569710695, 'reg_lambda': 0.507895191896573, 'n_jobs': 4, 'missing': -999}


C:\Users\yseon\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	train-rmse:4.30073	val-rmse:4.26096
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 30 rounds.
[30]	train-rmse:3.22956	val-rmse:3.2261
[60]	train-rmse:2.46783	val-rmse:2.5058
[90]	train-rmse:1.93305	val-rmse:2.00637
[120]	train-rmse:1.56225	val-rmse:1.66874
[150]	train-rmse:1.30984	val-rmse:1.4467
[180]	train-rmse:1.13971	val-rmse:1.30002
[210]	train-rmse:1.02415	val-rmse:1.20437
[240]	train-rmse:0.941445	val-rmse:1.13691
[270]	train-rmse:0.883915	val-rmse:1.09102
[300]	train-rmse:0.838485	val-rmse:1.05678
[330]	train-rmse:0.80335	val-rmse:1.03149
[360]	train-rmse:0.774779	val-rmse:1.01156
[390]	train-rmse:0.752161	val-rmse:0.996394
[420]	train-rmse:0.731002	val-rmse:0.982614
[450]	train-rmse:0.713814	val-rmse:0.971621
[480]	train-rmse:0.699954	val-rmse:0.963203
[510]	train-rmse:0.689183	val-rmse:0.957331
[540]	train-rmse:0.677723	val-rmse:0.952
[570]	train-rmse:0.669371	val-rmse:0.947805
[600]	train-

[2940]	train-rmse:0.487416	val-rmse:0.829351
[2970]	train-rmse:0.486613	val-rmse:0.82929
[2999]	train-rmse:0.485541	val-rmse:0.829161
[0]	train-rmse:4.28794	val-rmse:4.28519
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 30 rounds.
[30]	train-rmse:3.21384	val-rmse:3.22044
[60]	train-rmse:2.44886	val-rmse:2.47558
[90]	train-rmse:1.91026	val-rmse:1.96026
[120]	train-rmse:1.53795	val-rmse:1.61434
[150]	train-rmse:1.2872	val-rmse:1.38879
[180]	train-rmse:1.11698	val-rmse:1.24308
[210]	train-rmse:1.00175	val-rmse:1.14915
[240]	train-rmse:0.921076	val-rmse:1.08752
[270]	train-rmse:0.862796	val-rmse:1.04613
[300]	train-rmse:0.816347	val-rmse:1.01481
[330]	train-rmse:0.783519	val-rmse:0.993964
[360]	train-rmse:0.755868	val-rmse:0.977499
[390]	train-rmse:0.732719	val-rmse:0.964946
[420]	train-rmse:0.712944	val-rmse:0.954148
[450]	train-rmse:0.694883	val-rmse:0.945259
[480]	train-rmse:0.679664	val-rmse:0.937829
[5

 16%|████████████▉                                                                   | 135/834 [03:14<16:43,  1.44s/it]

In [ ]:
# def lgb_best_params(X, y,opt_params,init_points=2, optimization_round=20, n_folds=3, random_seed=0, cv_estimators=1000):
    
#     # prepare dataset
#     categorical_features = ["building_id", "site_id", "meter", "primary_use", "is_holiday", "weekend"]
#     train_data = lgb.Dataset(data=X, label=y, categorical_feature = categorical_features, free_raw_data=False)
    
#     def lgb_run(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight,learning_rate):
#         params = {"boosting": "gbdt",'application':'regression','num_iterations':cv_estimators, 'early_stopping_round':int(cv_estimators/5), 'metric':'rmse'}
#         params["num_leaves"] = int(round(num_leaves))
#         params['feature_fraction'] = max(min(feature_fraction, 1), 0)
#         params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
#         params['max_depth'] = int(round(max_depth))
#         params['lambda_l1'] = max(lambda_l1, 0)
#         params['lambda_l2'] = max(lambda_l2, 0)
#         params['min_split_gain'] = min_split_gain
#         params['min_child_weight'] = min_child_weight
#         params['learning_rate'] = learning_rate
#         cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=False, verbose_eval =cv_estimators, metrics=['rmse'])
#         return -min(cv_result['rmse-mean'])
    
#     params_finder = BayesianOptimization(lgb_run, opt_params, random_state=2021)
#     # optimize
#     params_finder.maximize(init_points=init_points, n_iter=optimization_round)

#     # return best parameters
#     return params_finder.max

In [ ]:
# #  number of fold
# fold = 3

# # Hyperparameter range
# params_range = {
#                 'num_leaves': (1000, 1280),
#                 'feature_fraction': (0.7, 0.9),
#                 'bagging_fraction': (0.8, 1),
#                 'max_depth': (10, 11),
#                 'lambda_l1': (2, 5),
#                 'lambda_l2': (2, 5),
#                 'min_split_gain': (0.001, 0.1),
#                 'min_child_weight': (5, 50),
#                 'learning_rate' : (.05,.07)
#                }

# # You can experiments with different estimators in a single execution. I'm using small numbers for demonstration purpose so you must change to high numbers.
# cv_estimators = [50,100,200]

# #n_iter: How many steps of bayesian optimization you want to perform. The more steps the more likely to find a good maximum you are.
# optimization_round = 10 # Simply, 10 models with different parameter will be tested. And the best one will be returned.

# #init_points: How many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.
# init_points = 2

# random_seed = 2010

In [ ]:
# best_params= []
# for cv_estimator in cv_estimators:
#     opt_params = lgb_best_params(features, target,params_range, init_points=init_points, optimization_round=optimization_round, n_folds=fold, random_seed=random_seed, cv_estimators=cv_estimator)
#     opt_params['params']['iteration'] = cv_estimator
#     opt_params['params']['fold'] = fold
#     opt_params['params']['rmse'] = opt_params['target']
#     best_params.append(opt_params['params'])

In [ ]:
# df_params = pd.DataFrame(best_params).reset_index()
# df_params = df_params[['iteration','fold','num_leaves','learning_rate','bagging_fraction',
#  'feature_fraction',
#  'lambda_l1',
#  'lambda_l2',
#  'max_depth',
#  'min_child_weight',
#  'min_split_gain',
#  'rmse']]
# df_params.to_csv('best_params.csv')